In [1]:
!pip install langchain openai weaviate-client ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.9/324.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
url="https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"

In [4]:
res = requests.get(url)

In [5]:
res.text

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [6]:
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

In [7]:
loader=TextLoader("./state_of_the_union.txt")

In [8]:
documents=loader.load()

In [9]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [10]:
chunks=text_splitter.split_documents(documents)

In [11]:
chunks[2].page_content

'Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. \n\nIn this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight. \n\nLet each of us here tonight in this Chamber send an unmistakable signal to Ukraine and to the world.'

In [12]:
len(chunks)


90

In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

In [15]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [16]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [17]:
client=weaviate.Client(
    embedded_options=EmbeddedOptions()
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 1213


In [18]:
vectorstore=Weaviate.from_documents(
    client=client,
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    by_text=False

)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [19]:
retriever=vectorstore.as_retriever()

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [22]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.2)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [24]:
prompt = ChatPromptTemplate.from_template(template)


In [25]:
rag_chain=(
    {"context":retriever, "question":RunnablePassthrough()}
          | prompt
          | llm
          | StrOutputParser()
)

In [26]:
res.text

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [27]:

questions=["what did the President say about Justic Breyer?",
           "What did the President say abou Intel's CEO?",
           "What did the President say about gun violence?"

           ]

In [28]:
groud_truths=[["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
              ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
              ["The president asked Congress to pass proven measures to reduce gun violence."]]

In [30]:
answer=[]
context=[]

In [31]:
for query in questions:
  answer.append(rag_chain.invoke(query))
  context.append([docs.page_content  for docs in retriever.get_relevant_documents(query)])




/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [32]:
answer

['The President honored Justice Breyer for his service and mentioned nominating Judge Ketanji Brown Jackson to continue his legacy.',
 "The President mentioned that Intel's CEO, Pat Gelsinger, is ready to increase their investment from $20 billion to $100 billion, which would be one of the biggest investments in manufacturing in American history.",
 'The President called for Congress to pass measures to reduce gun violence, including universal background checks and banning assault weapons and high-capacity magazines. He also emphasized the need to crack down on gun trafficking and ghost guns.']

In [33]:
context

[['Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.',
  'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  'A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Borde

In [34]:
# To dict
data = {
    "question": questions,
    "answer": answer,
    "contexts": context,
    "ground_truths": groud_truths
}




In [35]:
from datasets import Dataset
dataset=Dataset.from_dict(data)

In [36]:
!pip install Dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.30
    Uninstalling SQLAlchemy-2.0.30:
      Successfully uninstalled SQLAlchemy-2.0.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.


In [37]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [38]:
result=evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy
    ]
)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [39]:
result

{'context_precision': 0.9352, 'context_recall': 1.0000, 'faithfulness': 0.8056, 'answer_relevancy': 0.8432}

In [40]:
df=result.to_pandas()

In [41]:
df

,question,answer,contexts,ground_truths,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,what did the President say about Justic Breyer?,The President honored Justice Breyer for his s...,"[Tonight, I’d like to honor someone who has de...",[The president said that Justice Breyer has de...,The president said that Justice Breyer has ded...,1.000000,1.0,1.000000,0.848967
1,What did the President say abou Intel's CEO?,"The President mentioned that Intel's CEO, Pat ...",[But that’s just the beginning. \n\nIntel’s CE...,[The president said that Pat Gelsinger is read...,The president said that Pat Gelsinger is ready...,1.000000,1.0,0.666667,0.772534
2,What did the President say about gun violence?,The President called for Congress to pass meas...,[And I ask Congress to pass proven measures to...,[The president asked Congress to pass proven m...,The president asked Congress to pass proven me...,0.805556,1.0,0.750000,0.908009


In [44]:
df.iloc[0,:]

question               what did the President say about Justic Breyer?
answer               The President honored Justice Breyer for his s...
contexts             [Tonight, I’d like to honor someone who has de...
ground_truths        [The president said that Justice Breyer has de...
ground_truth         The president said that Justice Breyer has ded...
context_precision                                                  1.0
context_recall                                                     1.0
faithfulness                                                       1.0
answer_relevancy                                              0.848967
Name: 0, dtype: object

In [45]:
df.iloc[0,:].answer

'The President honored Justice Breyer for his service and mentioned nominating Judge Ketanji Brown Jackson to continue his legacy.'

In [46]:
#data
#this is my actual query
df.iloc[0,:].question

'what did the President say about Justic Breyer?'

In [47]:
#predicted
#generation of the model
df.iloc[0,:].answer

'The President honored Justice Breyer for his service and mentioned nominating Judge Ketanji Brown Jackson to continue his legacy.'

In [48]:
#actual
#actual answer(manually we have written this answer)(this could be llm generated)
df.iloc[0,:].ground_truth

'The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.'

In [ ]:
#average value
#retrieval result(context)
df.iloc[0,:].contexts[0]

'Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.'

In [49]:
from datasets import Dataset
from ragas.metrics import answer_similarity
from ragas import evaluate


data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_similarity])
score.to_pandas()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_similarity
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.996372
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.924343


In [50]:
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_correctness])
score.to_pandas()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_correctness
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.749093
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.981086


In [ ]:
#other frameworks
"""
raga:https://docs.ragas.io/en/latest/concepts/metrics/context_recall.html
TrueLence:https://www.trulens.org/trulens_eval/getting_started/core_concepts/rag_triad/
ares:https://github.com/stanford-futuredata/ARES?tab=readme-ov-file#section3
"""